In [22]:
# Import findspark and initialize.
import findspark
findspark.init()

In [23]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [24]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv", header=True, inferSchema=True))

25/03/10 14:49:42 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


TypeError: SparkFiles.get() got an unexpected keyword argument 'header'

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

NameError: name 'home_sales_df' is not defined

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_four_bedroom = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
""")
print("Average price for a four-bedroom house sold per year:")
avg_price_four_bedroom.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['year ASC NULLS FIRST], true
+- 'Aggregate ['year], ['YEAR('date) AS year#1, 'ROUND('AVG('price), 2) AS avg_price#2]
   +- 'Filter ('bedrooms = 4)
      +- 'UnresolvedRelation [home_sales], [], false


In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_three_bed_bath = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")
print("Average price of a home for each year built (3 Bed, 3 Bath):")
avg_price_three_bed_bath.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['date_built ASC NULLS FIRST], true
+- 'Aggregate ['date_built], ['date_built, 'ROUND('AVG('price), 2) AS avg_price#3]
   +- 'Filter (('bedrooms = 3) AND ('bathrooms = 3))
      +- 'UnresolvedRelation [home_sales], [], false


In [25]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_three_bed_bath_floors = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft-living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")
print("Average price of a home for each year built (3 Bed, 3 Bath, 2 floors, under 2000sqft):")
avg_price_three_bed_bath_floors.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['date_built ASC NULLS FIRST], true
+- 'Aggregate ['date_built], ['date_built, 'ROUND('AVG('price), 2) AS avg_price#4]
   +- 'Filter ((('bedrooms = 3) AND ('bathrooms = 3)) AND (('floors = 2) AND (('sqft - 'living) >= 2000)))
      +- 'UnresolvedRelation [home_sales], [], false


In [26]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_price_view_rating = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales
    GROUP BY date_built
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
print("Average price of a home per 'view' rating (under $350,000):")
avg_price_view_rating.show()
print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['view DESC NULLS LAST], true
+- 'UnresolvedHaving ('avg_price >= 350000)
   +- 'Aggregate ['date_built], ['view, 'ROUND('AVG('price), 2) AS avg_price#5]
      +- 'UnresolvedRelation [home_sales], [], false


In [27]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [home_sales], [], false


In [28]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("home_sales")
print("Is 'home_sales' cached?", is_cached)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [home_sales], [], false


In [29]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_price_view_rating_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales
    GROUP BY date_built
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
print("Average price of a home per 'view' rating (cached, under $350,000):")
avg_price_view_rating.show()
print("--- %s seconds ---" % (time.time() - start_time))


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['view DESC NULLS LAST], true
+- 'UnresolvedHaving ('avg_price >= 350000)
   +- 'Aggregate ['date_built], ['view, 'ROUND('AVG('price), 2) AS avg_price#6]
      +- 'UnresolvedRelation [home_sales], [], false


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet", mode="overwrite")

NameError: name 'home_sales_df' is not defined

In [33]:
# 11. Read the formatted parquet data.
home_sales_partitioned_df = spark.read.parquet("home_sales_partitioned.parquet")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/admin/Documents/GitHub/Home_Sales/module_22_code/home_sales_partitioned.parquet.

In [34]:
# 12. Create a temporary table for the parquet data.
home_sales_partitioned_df.creeateOrReplaceTempView("home_sales_partitioned")

NameError: name 'home_sales_partitioned_df' is not defined

In [35]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
avg_price_view_rating_partitioned = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales_paritioned 
    GROUP BY view
    HAVING  avg_price >= 350000
    ORDER BY view DESC
""")
print("Average price of a home per 'view' rating (partitioned, >= $350,000):")
avg_price_view_rating_partitioned.show()
print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales_paritioned` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 9;
'Sort ['view DESC NULLS LAST], true
+- 'UnresolvedHaving ('avg_price >= 350000)
   +- 'Aggregate ['view], ['view, 'ROUND('AVG('price), 2) AS avg_price#7]
      +- 'UnresolvedRelation [home_sales_paritioned], [], false


In [36]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [home_sales], [], false


In [37]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [home_sales], [], false


In [ ]:
# End Session
spark.stop()